In [ ]:
from abc import ABC, abstractmethod
from IPython.display import display
from IPython.display import Markdown
import os
import google.generativeai as genai
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


class AIModel:
  @abstractmethod
  def train(self):
    pass

  @abstractmethod
  def prompt(self,text):
    pass



def save_text_to_file(file_path, text):
    """Saves text to a file, creating the directory if necessary."""

    os.makedirs(os.path.dirname(file_path), exist_ok=True)  # Create directory if needed

    with open(file_path, "w") as file:
        file.write(text)

# # Example usage:
# file_path = "path/to/your/file.txt"  # Replace with the desired file path
# text = "This is the text to be saved."

# save_text_to_file(file_path, text)

# print("Text saved successfully!")





class OpenAIModel(AIModel):
    def __init__(self,keys,model_type):
      self.keys = keys
      self.model_type = model_type
    def train(self):
      print('Nothing')
    def prompt(self,text):
      OPENAI_API_KEY0 = self.keys[0]
      client = OpenAI(api_key=OPENAI_API_KEY0)
      completion = client.chat.completions.create(
      model=self.model_type,
      messages=[
          {"role": "system", "content": text['prompt1']},
          {"role": "user", "content": text['prompt2']}
      ]
      )
      answer = completion.choices[0].message.content
      # save_text_to_file('/prompts.txt',text[prompt2].encode())
      # save_text_to_file('/outputs.txt',answer)
      return answer


class GeminiAIModel(AIModel):
  def __init__(self,keys,model_type,retention=True):
    self.keys = keys
    self.model_type = model_type
    self.retention = retention
    self.models=[]
    self.iter = 0
    for i in self.keys:
      genai.configure(api_key=i)
      model = genai.GenerativeModel(self.model_type)
      chat = model.start_chat(history=[]) if self.retention  else  model.generate_content()
      self.models.append(chat)

  def train(self,one_shot_prompt):
    index = 0
    for i in self.models:
      i.send_message(one_shot_prompt)
      print(index)
      index += 1

  def prompt(self,text):
    response = self.models[self.iter % len(self.models)].send_message(text)
    self.iter += 1
    # save_text_to_file('/prompts.txt',text.encode())
    # save_text_to_file('/outputs.txt',response.text)
    return response.text